In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
from datetime import date
import math

# constants
this_year = date.today().year
this_year_p1 = math.floor(this_year / 100)  # first two digits
this_year_p2 = this_year % 100  # last two digits

lfall.select(list_id_cols).with_columns(  # slice first two digits as year_p2
  pl.when(pl.col("ICNUMBER").str.contains(r"^\d{12}$"))
  .then(True)
  .otherwise(False)
  .alias("valid_ic_digits"),
  pl.col("ICNUMBER").str.slice(0, 2).cast(pl.Int16).alias("year_p2"),
).with_columns(  # calculate first two digits of birth year from IC
  pl.when(pl.col("year_p2") > this_year_p2)
  .then(this_year_p1 - 1)
  .otherwise(this_year_p1)
  .cast(pl.Utf8)
  .alias("year_p1"),
).with_columns(
  pl.concat_str(  # concat into full date string
    pl.col("year_p1"),
    pl.col("ICNUMBER").str.slice(0, 2),
    pl.lit("-"),
    pl.col("ICNUMBER").str.slice(2, 2),
    pl.lit("-"),
    pl.col("ICNUMBER").str.slice(4, 2),
  ).alias("datebirth_from_ic")
).drop(["year_p1", "year_p2"]).with_columns(
  pl.col("datebirth_from_ic").str.to_date(strict=False)
).with_columns(
  pl.when(pl.col("datebirth_from_ic").is_null())
  .then(False)
  .otherwise(True)
  .alias("valid_ic_date")
).with_columns(
  pl.all_horizontal(pl.col("valid_ic_digits"), pl.col("valid_ic_date")).alias(
    "valid_ic"
  )
).filter(pl.col("valid_ic") == False).collect()


In [ ]:
lfall.columns

0 - No such habit
1- habit currently practiced
2 - past habit now has stopped (minimum 6 months)

In [ ]:
str_habit_false = "0 - No such habit"
lfall.select(#*list_id_cols,
    "TOBACCO",
    "TOBACCO_ADVISED",
    "TOBACCO QUIT",
    "REFERRAL TO QUIT SERVICES",
    "DATE REFERRED QUIT SER",
).filter((pl.col("REFERRAL TO QUIT SERVICES") == True) & (pl.col("DATE REFERRED QUIT SER").is_null())).collect()
